In [1]:
from selenium import webdriver
from bs4 import BeautifulSoup
import os
import requests

In [2]:
cera_url = "https://www.coursera.org"
course_hrefs = {}
lecture_hrefs = {}
week_hrefs = {}
weeks = 0

In [3]:
#Open web browser
driver = webdriver.Chrome()
driver.maximize_window()
driver.implicitly_wait(5)
driver.get(cera_url)

In [5]:
#LOG IN
driver.find_element_by_link_text("LOG IN").click()

In [6]:
#Find the input fields and the submit button
uname = driver.find_element_by_xpath("//input[@name='email']")
passwd = driver.find_element_by_xpath("//input[@name='password']")
submit = driver.find_element_by_xpath("//form//button")

In [7]:
#form = driver.find_element_by_xpath("//form[@name='login']")
#form.get_attribute('action')

In [8]:
#Send the keys to the input fields
uname.send_keys("avisektalks@gmail.com")
passwd.send_keys("Avisek@4326")

In [9]:
submit.click()

TimeoutException: Message: timeout
  (Session info: chrome=68.0.3440.106)
  (Driver info: chromedriver=2.37.544315 (730aa6a5fdba159ac9f4c1e8cbc59bf1b5ce12b7),platform=Linux 4.15.0-33-generic x86_64)


In [10]:
driver.find_element_by_xpath("//button[@class='nostyle dropdown']")

<selenium.webdriver.remote.webelement.WebElement (session="74a2e1ba4ecab8673bb141168a4ab676", element="0.5111743110714826-1")>

In [11]:
driver.find_element_by_xpath("//button[@class='nostyle dropdown']").click()

In [12]:
driver.execute_script("window.scrollTo(0,0)")

In [13]:
soup_lastActive = BeautifulSoup(driver.page_source,'html.parser')

In [14]:
#section_list = soup_lastActive.find_all('section',attrs={"class":"rc-CourseCard with-padding".split()})
modified_section_list = soup_lastActive.find_all('section',attrs={"class":"rc-CourseCard with-padding card-rich-interaction".split()})

In [15]:
for section in modified_section_list:
    print(section.find('h4',attrs={"class":"headline-1-text"}).text)

Machine Learning
Neural Networks for Machine Learning
Introduction to Data Science in Python
Neural Networks and Deep Learning
Neural Networks and Deep Learning
Improving Deep Neural Networks: Hyperparameter tuning, Regularization and Optimization
Structuring Machine Learning Projects
Convolutional Neural Networks
Sequence Models


In [16]:
for section in modified_section_list:
    print(section.find('h4').text)
    check_enrollment = section.find('div',attrs={"class":"rc-CourseEnrollButton"})
    #print(check_enrollment)
    print(None if check_enrollment == None else check_enrollment.find('span').text )

Machine Learning
None
Neural Networks for Machine Learning
None
Introduction to Data Science in Python
None
Neural Networks and Deep Learning
Go to Course
Neural Networks and Deep Learning
None
Improving Deep Neural Networks: Hyperparameter tuning, Regularization and Optimization
Go to Course
Structuring Machine Learning Projects
Enroll
Convolutional Neural Networks
Enroll
Sequence Models
Enroll


In [17]:
for section in modified_section_list:
        check_enrollment = section.find('div',attrs={"class":"rc-CourseEnrollButton"})
        enroll_text = check_enrollment.find('span').text if check_enrollment!=None else 'None'
        print(enroll_text)
        if enroll_text != 'Enroll':
            name = section.find('h4', attrs={"class": "headline-1-text"}).text
            href_link = section.find("a", href=True)['href']
            course_hrefs[name] = href_link           

None
None
None
Go to Course
None
Go to Course
Enroll
Enroll
Enroll


In [18]:
course_hrefs

{'Improving Deep Neural Networks: Hyperparameter tuning, Regularization and Optimization': '/learn/deep-neural-network/home/welcome',
 'Introduction to Data Science in Python': '/learn/python-data-analysis/home/welcome',
 'Machine Learning': '/learn/machine-learning/home/welcome',
 'Neural Networks and Deep Learning': '/learn/neural-networks-deep-learning/home/welcome',
 'Neural Networks for Machine Learning': '/learn/neural-networks/home/welcome'}

In [19]:
course_url = cera_url+course_hrefs['Machine Learning']
course_url

'https://www.coursera.org/learn/machine-learning/home/welcome'

In [20]:
driver.get(course_url)

In [21]:
soup = BeautifulSoup(driver.page_source,'html.parser')
div_weeks = soup.find('div',attrs={'class':'rc-NavigationDrawer'})
weeks = div_weeks.find_all('a',attrs={'class' : "rc-NavigationDrawerLink headline-1-text horizontal-box \
                                                   rc-WeekNavigationItem".split()})

In [22]:
len(weeks)

11

In [31]:
driver.find_element_by_link_text("Week "+str(1)).click()

In [32]:
soup  = BeautifulSoup(driver.page_source,'html.parser')

In [33]:
lesson_divs = soup.find_all('div',{"class":"rc-NamedItemList"})

In [34]:
len(lesson_divs)

8

In [35]:
lessons = [li for li in [div.find_all('li') for div in lesson_divs if div.find('h4').text != 'Review']]

In [36]:
lessons = []
for div in lesson_divs:
    if div.find('h4').text != "Review":
        for li in div.find_all('li'):
            lessons.append(li)
len(lessons)

33

In [37]:
for c,lesson in enumerate(lessons):
    lesson_div = lesson.find('div',attrs={'class':'rc-WeekItemName headline-1-text'.split()})
    lesson_div.find('span').replace_with('')
    lecture_link = lesson.find('a')['href']
    if "/lecture/" in lecture_link:
        lecture_name = lesson_div.text
    lecture_hrefs[(str(c+1)+". "+lecture_name)] = cera_url + lecture_link

In [38]:
lecture_hrefs

{'1. Welcome to Machine Learning!': 'https://www.coursera.org/learn/machine-learning/lecture/zcAuT/welcome-to-machine-learning',
 '10. Unsupervised Learning': 'https://www.coursera.org/learn/machine-learning/supplement/1O0Bk/unsupervised-learning',
 '11. Unsupervised Learning': 'https://www.coursera.org/learn/machine-learning/supplement/JBzFm/who-are-mentors',
 '12. Unsupervised Learning': 'https://www.coursera.org/learn/machine-learning/supplement/dJCBb/get-to-know-your-classmates',
 '13. Unsupervised Learning': 'https://www.coursera.org/learn/machine-learning/supplement/gBboB/frequently-asked-questions',
 '14. Model Representation': 'https://www.coursera.org/learn/machine-learning/lecture/db3jS/model-representation',
 '15. Model Representation': 'https://www.coursera.org/learn/machine-learning/supplement/cRa2m/model-representation',
 '16. Cost Function': 'https://www.coursera.org/learn/machine-learning/lecture/rkTp3/cost-function',
 '17. Cost Function': 'https://www.coursera.org/lear

In [39]:
driver.get(lecture_hrefs[list(lecture_hrefs.keys())[0]])

In [40]:
soup = BeautifulSoup(driver.page_source,'html.parser')

In [41]:
vid_link = soup.find('video').find('source')['src']

In [42]:
vid_link

'https://d3c33hcgiwev3.cloudfront.net/qHTMkA4fEeW2rSIAC2yC6g.processed/full/540p/index.mp4?Expires=1536105600&Signature=MKVs68NBUlu5ODk1GOaf9ZbWTLXItUV27xOBwEGz~ZMkFA6tsoN8rMbqoyraFzxSftZ5Pi2KglrwCtrXMEJzTJsRJHkOT9PPPlbqw95woI~fe2hAZCfLKhbHSfUYpF61hYtNSD6JM8L596DkvaoJ8gLHkUW0VcdQ6Nkz53kbuBM_&Key-Pair-Id=APKAJLTNE6QMUY6HBC5A'

In [56]:
#driver.get(vid_link)

In [43]:
_params = {'Expires':'1536105600',
          'Signature':'MKVs68NBUlu5ODk1GOaf9ZbWTLXItUV27xOBwEGz~ZMkFA6tsoN8rMbqoyraFzxSftZ5Pi2KglrwCtrXMEJzTJsRJHkOT9PPPlbqw95woI~fe2hAZCfLKhbHSfUYpF61hYtNSD6JM8L596DkvaoJ8gLHkUW0VcdQ6Nkz53kbuBM_',
          'Key-Pair-Id':'APKAJLTNE6QMUY6HBC5A'}

In [48]:
vid = requests.get("https://d3c33hcgiwev3.cloudfront.net/qHTMkA4fEeW2rSIAC2yC6g.processed/full/540p/index.mp4",params=_params,stream=True)
print(vid.url)

https://d3c33hcgiwev3.cloudfront.net/qHTMkA4fEeW2rSIAC2yC6g.processed/full/540p/index.mp4?Expires=1536105600&Signature=MKVs68NBUlu5ODk1GOaf9ZbWTLXItUV27xOBwEGz~ZMkFA6tsoN8rMbqoyraFzxSftZ5Pi2KglrwCtrXMEJzTJsRJHkOT9PPPlbqw95woI~fe2hAZCfLKhbHSfUYpF61hYtNSD6JM8L596DkvaoJ8gLHkUW0VcdQ6Nkz53kbuBM_&Key-Pair-Id=APKAJLTNE6QMUY6HBC5A


In [49]:
f = open('index.mp4','wb')
f.write(vid.content)

4819806